# 0.0 IMPORTS

In [3]:
import pandas as pd

# 1.0 CARREGANDO DADOS

In [139]:
df_transactions_raw = pd.read_csv("data/202110_CPGF.csv", sep=';', encoding='utf8', decimal=",")

In [5]:
df_transactions_raw.sample(3)

,CÓDIGO ÓRGÃO SUPERIOR,NOME ÓRGÃO SUPERIOR,CÓDIGO ÓRGÃO,NOME ÓRGÃO,CÓDIGO UNIDADE GESTORA,NOME UNIDADE GESTORA,ANO EXTRATO,MÊS EXTRATO,CPF PORTADOR,NOME PORTADOR,CNPJ OU CPF FAVORECIDO,NOME FAVORECIDO,TRANSAÇÃO,DATA TRANSAÇÃO,VALOR TRANSAÇÃO
7168,53000,Ministério do Desenvolvimento Regional,29214,Companhia Brasileira de Trens Urbanos,275058,CIA.BRASILEIRA DE TRENS URBANOS-STU/BH,2021,10,***.311.446-**,MARCELO DE OLIVEIRA CRUZ,1.057350e+13,MERCADOPAGO.COM REPRESENTACOES LTDA.,COMPRA A/V - R$ - APRES,17/09/2021,56.0
2107,52000,Ministério da Defesa,52221,Indústria de Material Bélico do Brasil,168004,INDUSTRIA DE MATERIAL BELICO DO BRASIL/FPV,2021,10,***.863.058-**,CELSO BENEDITO MOREIRA,-2.000000e+00,NAO SE APLICA,SAQUE CASH/ATM BB,26/08/2021,560.0
140,22000,"Ministério da Agricultura, Pecuária e Abastec",22202,Empresa Brasileira de Pesquisa Agropecuária,135033,EMBRAPA/CNPUV,2021,10,***.906.560-**,ILDOMAR ENGROFF DOS SANTOS,-2.000000e+00,NAO SE APLICA,SAQUE CASH/ATM BB,21/09/2021,175.0


# 2.0 DESCRIÇÃO DOS DADOS

In [6]:
df1 = df_transactions_raw.copy()

## 2.1 Dimensão dos dados

In [7]:
df1.shape

(8621, 15)

## 2.1 Tipos de dados

In [8]:
df1.dtypes

CÓDIGO ÓRGÃO SUPERIOR       int64
NOME ÓRGÃO SUPERIOR        object
CÓDIGO ÓRGÃO                int64
NOME ÓRGÃO                 object
CÓDIGO UNIDADE GESTORA      int64
NOME UNIDADE GESTORA       object
ANO EXTRATO                 int64
MÊS EXTRATO                 int64
CPF PORTADOR               object
NOME PORTADOR              object
CNPJ OU CPF FAVORECIDO    float64
NOME FAVORECIDO            object
TRANSAÇÃO                  object
DATA TRANSAÇÃO             object
VALOR TRANSAÇÃO           float64
dtype: object

## 2.2 Altera tipos

In [76]:
# Altera o tipo do campo 'CNPJ OU CPF FAVORECIDO' de float para string
df1['CNPJ OU CPF FAVORECIDO'] = df1['CNPJ OU CPF FAVORECIDO'].astype(str)

In [88]:
# Altera o tipo do campo 'DATA TRANSAÇÃO' de object para datetime
df1['DATA TRANSAÇÃO'] = pd.to_datetime(df1['DATA TRANSAÇÃO'])

In [89]:
df1.dtypes

CÓDIGO ÓRGÃO SUPERIOR              int64
NOME ÓRGÃO SUPERIOR               object
CÓDIGO ÓRGÃO                       int64
NOME ÓRGÃO                        object
CÓDIGO UNIDADE GESTORA             int64
NOME UNIDADE GESTORA              object
ANO EXTRATO                        int64
MÊS EXTRATO                        int64
CPF PORTADOR                      object
NOME PORTADOR                     object
CNPJ OU CPF FAVORECIDO            object
NOME FAVORECIDO                   object
TRANSAÇÃO                         object
DATA TRANSAÇÃO            datetime64[ns]
VALOR TRANSAÇÃO                  float64
dtype: object

## 2.3 Verifica NA

In [26]:
df1.isna().sum()

CÓDIGO ÓRGÃO SUPERIOR        0
NOME ÓRGÃO SUPERIOR          0
CÓDIGO ÓRGÃO                 0
NOME ÓRGÃO                   0
CÓDIGO UNIDADE GESTORA       0
NOME UNIDADE GESTORA         0
ANO EXTRATO                  0
MÊS EXTRATO                  0
CPF PORTADOR              2373
NOME PORTADOR                0
CNPJ OU CPF FAVORECIDO       0
NOME FAVORECIDO              0
TRANSAÇÃO                    0
DATA TRANSAÇÃO            2306
VALOR TRANSAÇÃO              0
dtype: int64

# 3.0 Questões

## 3.1 Utilidades

In [137]:
def format_to_brl_currency(amount):
    # Source: https://www.kite.com/python/answers/how-to-format-currency-in-python
    thousands_separator = "."
    fractional_separator = ","
    currency = "R${:,.2f}".format(amount)
    if thousands_separator == ".":
        main_currency, fractional_currency = currency.split(".")[0], currency.split(".")[1]
        new_main_currency = main_currency.replace(",", ".")
        currency = new_main_currency + fractional_separator + fractional_currency
    return currency

## 3.2 Respostas

### A – Com suas palavras explique o que é lavagem de dinheiro.

Lavagem de dinheiro é o processo que tem por objetivo tornar o dinheiro de origem ilícita em dinheiro de origem lícita.

### B – O que é Cartão de Pagamento do Governo Federal (CPGF), e qual a sua finalidade.

De acordo com o site do Portal da Transparência[[1]](#Referências), o Cartão de Pagamento do Governo Federal é:
> "(...) um meio de pagamento utilizado pelo governo que funciona de forma similar ao cartão de crédito que utilizamos em nossas vidas, porém dentro de limites e regras específicas."

Ainda de acordo com [1] a finalidade do CPGF é:
> (...) para pagamentos de despesas próprias, que possam ser enquadradas como suprimento de fundos.

### C – Quem pode utilizar o CPGF?

  Concedido ao **servidor** para pagamento de despesas. [[1][4]](#Referências)

### D – Qual a URL onde é possível fazer o download dos arquivos do CPGF?

| Dados do CPGF para download | https://www.portaltransparencia.gov.br/download-de-dados/cpgf |
|-----------------------------|---------------------------------------------------------------|

### E – Qual a URL da paǵina com a descrição dos campos (ou dicionário de dados) da CPGF?

| Dicionário de dados | https://www.portaldatransparencia.gov.br/pagina-interna/603393-dicionario-de-dados-cpgf |
|---------------------|-----------------------------------------------------------------------------------------|

### F – É possível identificar o nome e o documento do portador do CPGF, em todas as movimentações ou há movimentações onde não é possível identificar o portador?

Não é possível identificar o nome e o documento do portador do CPGF em todas as movimentações, pois há movimentações em que o nome do portador é *'Sigiloso'* ou *'SEM INFORMAÇÂO'* e há registros com o campo documento *vazio*.

In [32]:
# Gera dataset que contém CPF PORTADOR como NA (não informado)
# Na coluna NOME PORTADOR verifica quais são os valores únicos existentes
df1[df1['CPF PORTADOR'].isna()]['NOME PORTADOR'].unique()

array(['SEM INFORMACAO', 'Sigiloso'], dtype=object)

In [33]:
df1.isna().sum()

CÓDIGO ÓRGÃO SUPERIOR        0
NOME ÓRGÃO SUPERIOR          0
CÓDIGO ÓRGÃO                 0
NOME ÓRGÃO                   0
CÓDIGO UNIDADE GESTORA       0
NOME UNIDADE GESTORA         0
ANO EXTRATO                  0
MÊS EXTRATO                  0
CPF PORTADOR              2373
NOME PORTADOR                0
CNPJ OU CPF FAVORECIDO       0
NOME FAVORECIDO              0
TRANSAÇÃO                    0
DATA TRANSAÇÃO            2306
VALOR TRANSAÇÃO              0
dtype: int64


### G – É possível identificar o Órgão do portador do CPGF?

É possível identificar o Órgão do portador do CPFG **através do campo *'NOME ÓRGÃO'***.

In [141]:
dfG = df1[['NOME PORTADOR', 'NOME ÓRGÃO']]

In [142]:
dfG.sample(2)

,NOME PORTADOR,NOME ÓRGÃO
7531,LEONARDO VIANNA MOHR,Instituto Chico Mendes de Conservação da Biodi...
528,ANTONIO VIRGINIO DE ALENCAR,Instituto Nacional de Colonização e Reforma Ag...


### H - Qual o nome do Órgão cujo código é 20402?

In [35]:
# Seleciona as colunas de interesse
dfH = df1[['CÓDIGO ÓRGÃO', 'NOME ÓRGÃO']]

In [39]:
# Filtra os registros nos quais o CÓDIGO ÓRGÃO seja igual a 20402
# Exibe apenas 1o registro do dataset
dfH[dfH['CÓDIGO ÓRGÃO'] == 20402].head(1)

,CÓDIGO ÓRGÃO,NOME ÓRGÃO
844,20402,Agência Espacial Brasileira


### I - É possível identificar o Nome e Documento (CNPJ) dos favorecidos pela utilização do CPGF?

É possível identificar o Nome e Documento (CNPJ) dos favorecidos **somente em parte dos dados**, pois há movimentações com status como: *NAO SE APLICA*, *SEM INFORMACAO* e *Sigiloso*.

In [78]:
# Seleciona as colunas de interesse
dfI = df1[['NOME FAVORECIDO', 'CNPJ OU CPF FAVORECIDO']]
dfI.sample(3)

,NOME FAVORECIDO,CNPJ OU CPF FAVORECIDO
4848,Sigiloso,-11.0
8230,JOSE ALEX DA SILVA,33297947802.0
5844,Sigiloso,-11.0


In [84]:
# Seleciona os registros onde o 'CNPJ OU CPF FAVORECIDO' possui valor negativo
# Foi observado que valores negativos indicam registros com informações adicionais
dfI = dfI[dfI['CNPJ OU CPF FAVORECIDO'].astype(float) < 0]

In [83]:
# Exibe o que foi preenchido no 'NOME FAVORECIDO' dos registros que
# possuem 'CNPJ OU CPF FAVORECIDO' com valor negativo
dfI['NOME FAVORECIDO'].unique()

array(['NAO SE APLICA', 'SEM INFORMACAO', 'Sigiloso'], dtype=object)

### J – É possível identificar a data e o valor das movimentações financeiras do CPGF, em todas as movimentações? Ou há movimentações onde não é possível identificar as datas e ou valores?

| Campo           | Observações                                                                                     |
|-----------------|-------------------------------------------------------------------------------------------------|
| DATA TRANSAÇÃO  | As movimentações que possuem informações protegidas por sigilo não informam a data da transação |
| VALOR TRANSAÇÃO | Todos os valores foram informados                                                               |

In [105]:
# Seleciona as colunas de interesse
dfJ = df1[['DATA TRANSAÇÃO', 'VALOR TRANSAÇÃO', 'NOME PORTADOR']]

In [111]:
# Exibe registros sem data da transação
dfJ.isna().sum()

DATA TRANSAÇÃO     2306
VALOR TRANSAÇÃO       0
NOME PORTADOR         0
dtype: int64

In [109]:
# Seleciona registros que não possuem DATA TRANSAÇÃO
dfJ = dfJ[dfJ['DATA TRANSAÇÃO'].isna() == True]

In [110]:
# Exibe os valores preenchidos em NOME PORTADOR para os registros sem informação de DATA TRANSAÇÃO
dfJ['NOME PORTADOR'].unique()

array(['Sigiloso'], dtype=object)

### K (código) – Qual a soma total das movimentações utilizando o CPGF?

| Total das movimentações utilizando o CPGF | R$ 5.619.007,95 |
|-------------------------------------------|-----------------|

In [122]:
# Seleciona coluna de interesse
dfK = df1[['VALOR TRANSAÇÃO']]

In [138]:
# Soma o campo 'VALOR TRANSAÇÃO' de todos os registros de movimentação.
format_to_brl_currency(sum(dfK['VALOR TRANSAÇÃO']))

'R$5.619.007,95'

### L (código) – Qual a soma das movimentações sigilosas?


| Total das movimentações sigilosas | R$ 3.108.731,15 |
|-----------------------------------|-----------------|

**Premissa:**

Movimentação Sigilosa: 
É aquela na qual o campo **NOME FAVORECIDO** é igual a **Sigiloso**.

In [126]:
# Cria dataframe com as movimentações que contem NOME FAVORECIDO == Sigiloso
df_sigilosas = df1[df1['NOME FAVORECIDO'] == 'Sigiloso']

In [127]:
# Soma o campo VALOR TRANSAÇÃO do dataframe de movimentações Sigilosas
total_movimentacoes_sigilosas = sum(df_sigilosas['VALOR TRANSAÇÃO'])
format_to_brl_currency(total_movimentacoes_sigilosas)

'R$3.108.731,15'

### M (código) – Qual o Órgão que mais realizou movimentações sigilosas no período e qual o valor (somado)?

| NOME ÓRGÃO               | TOTAL TRANSAÇÕES SIGILOSAS   |
|--------------------------|:----------------------------:|
| Presidência da República | R$ 1.699.751,04              |

In [14]:
# Cria dataframe com as colunas de interesse a partir do dataframe de movimentações sigilosas
dfM = df_sigilosas[['NOME ÓRGÃO', 'VALOR TRANSAÇÃO']]
# Faz a soma do VALOR TRANSAÇÃO agrupando por NOME ÓRGÃO
dfM = dfM.groupby(['NOME ÓRGÃO']).agg({'VALOR TRANSAÇÃO' : sum})
# Obtem Órgão com maior somatório de movimentações sigilosas
dfM.nlargest(1, 'VALOR TRANSAÇÃO')

,VALOR TRANSAÇÃO
NOME ÓRGÃO,
Presidência da República,1699751.04


### N (código) – Qual o nome do portador que mais realizou saques no período? Qual a soma dos saques realizada por ele? Qual o nome do Órgão desse portador?


| NOME PORTADOR         | NOME ÓRGÃO                                              | TOTAL SAQUES |
|-----------------------|---------------------------------------------------------|--------------|
| RAFAEL FERREIRA COSTA | Instituto Chico Mendes de Conservação da Biodiversidade | R$ 24.520,0  |

In [134]:
# Cria dataframe apenas com as colunas de interesse ['NOME ÓRGÃO', 'NOME PORTADOR', 'TRANSAÇÃO', 'VALOR TRANSAÇÃO']
dfN = df1[['NOME ÓRGÃO', 'NOME PORTADOR', 'TRANSAÇÃO', 'VALOR TRANSAÇÃO']]

In [136]:
# Identifica valores de preenchimento do campo TRANSAÇÃO
dfN['TRANSAÇÃO'].unique()

array(['SAQUE CASH/ATM BB', 'COMPRA A/V - R$ - APRES',
       'SAQUE - INT$ - APRES', 'COMPRA A/V - INT$ - APRES',
       'Informações protegidas por sigilo',
       'COMP A/V-SOL DISP C/CLI-R$ ANT VENC'], dtype=object)

**Premissa**: Considerou-se um saque todos os movimentos que continham no campo 'TRANSAÇÃO' a palavra 'SAQUE'.

In [131]:
# Filtra movimentações que contenham "SAQUE" na coluna 'TRANSAÇÃO
dfN = dfN[dfN['TRANSAÇÃO'].str.contains('SAQUE')]

In [132]:
# Faz a soma das transações realizadas agrupando por PORTADOR e depois por NOME ÓRGÃO
dfN = dfN.groupby(['NOME PORTADOR', 'NOME ÓRGÃO']).agg({'VALOR TRANSAÇÃO' : sum})

In [133]:
# Obtem o PORTADOR com o maior somatório de VALOR TRANSAÇÃO 
dfN.nlargest(1, 'VALOR TRANSAÇÃO')

,,VALOR TRANSAÇÃO
NOME PORTADOR,NOME ÓRGÃO,
RAFAEL FERREIRA COSTA,Instituto Chico Mendes de Conservação da Biodiversidade,24520.0


### O (código) – Qual o nome do favorecido que mais recebeu compras realizadas utilizando o CPGF?


O favorecido, identificável, que mais recebeu compras realizadas utilizando o CPGF foi o MERCADOPAGO.COM REPRESENTACOES LTDA.

In [19]:
# Cria dataframe apenas com as colunas de interesse ['NOME FAVORECIDO', 'VALOR TRANSAÇÃO']
dfO = df1[['NOME FAVORECIDO', 'VALOR TRANSAÇÃO']]

In [20]:
# Agrupa soma das transações recebidas por FAVORECIDO e depois por NOME ÓRGÃO
dfO = dfO.groupby(['NOME FAVORECIDO']).agg({'VALOR TRANSAÇÃO' : sum})

In [21]:
# Obtem o FAVORECIDO com o maior somatório de VALOR TRANSAÇÃO 
dfO.nlargest(4, 'VALOR TRANSAÇÃO')

,VALOR TRANSAÇÃO
NOME FAVORECIDO,
Sigiloso,3108731.15
NAO SE APLICA,367504.96
SEM INFORMACAO,67421.29
MERCADOPAGO.COM REPRESENTACOES LTDA.,60694.21


### P - Descreva qual a abordagem utilizada para desenvolver o código para os ítens de K a O.

- K
  - Somar o campo 'VALOR TRANSAÇÃO' de todos os registros de movimentação.
- L
  > **Premissa:**
Movimentação Sigilosa: É aquela na qual o campo NOME FAVORECIDO é igual a Sigiloso.

  - Criar dataframe com as movimentações que contem NOME FAVORECIDO == Sigiloso
Somar o campo VALOR TRANSAÇÃO do dataframe de movimentações Sigilosas
- M
  - Criar dataframe com as colunas de interesse a partir do dataframe de movimentações sigilosas
  - Fazer a soma do VALOR TRANSAÇÃO agrupando por NOME ÓRGÃO
  - Obter Órgão com maior somatório de movimentações sigilosas

- N
    > **Premissa**: Considerou-se um saque todos os movimentos que continham no campo 'TRANSAÇÃO' a palavra 'SAQUE'.
    
  - Criar dataframe apenas com as colunas de interesse ['NOME ÓRGÃO', 'NOME PORTADOR', 'TRANSAÇÃO', 'VALOR TRANSAÇÃO']
  - Identificar valores de preenchimento do campo TRANSAÇÃO
  - Filtrar movimentações que contenham "SAQUE" na coluna 'TRANSAÇÃO
  - Fazer a soma das transações realizadas agrupando por PORTADOR e depois por NOME ÓRGÃO
  - Obter o PORTADOR com o maior somatório de VALOR TRANSAÇÃO 
- O
  - Criar dataframe apenas com as colunas de interesse ['NOME FAVORECIDO', 'VALOR TRANSAÇÃO']
  - Agrupar soma das transações recebidas por FAVORECIDO e depois por NOME ÓRGÃO
  - Obter o FAVORECIDO com o maior somatório de VALOR TRANSAÇÃO


# 4.0 Referências

1. https://www.portaltransparencia.gov.br/pagina-interna/603242-cartao-de-pagamento-do-governo-federal
2. https://www.gov.br/compras/pt-br/agente-publico/orientacoes-no-combate-a-covid-19/midias/suprimento-de-fundos.pdf
3. http://www.planalto.gov.br/ccivil_03/_ato2004-2006/2005/decreto/D5355compilado.htm
4. https://www.gov.br/cgu/pt-br/centrais-de-conteudo/publicacoes/orientacoes-aos-gestores/arquivos/suprimento-de-fundos-e-cartao-de-pagamento.pdf